In [1]:
import os
import multiprocessing
import warnings

# from tqdm import tqdm
import numpy as np
from scipy import stats
import pandas as pd
import librosa

import utils


def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()




In [2]:
features = pd.Series(index=columns(), dtype=np.float32)
features

feature      statistics  number
chroma_cens  kurtosis    01       NaN
                         02       NaN
                         03       NaN
                         04       NaN
                         05       NaN
                         06       NaN
                         07       NaN
                         08       NaN
                         09       NaN
                         10       NaN
                         11       NaN
                         12       NaN
             max         01       NaN
                         02       NaN
                         03       NaN
                         04       NaN
                         05       NaN
                         06       NaN
                         07       NaN
                         08       NaN
                         09       NaN
                         10       NaN
                         11       NaN
                         12       NaN
             mean        01       NaN
                  

In [3]:
def sub_features(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

In [5]:
final = pd.DataFrame(index=[1,2,3,4,5,6],
                            columns=columns(), dtype=np.float32)

In [6]:
final

feature    chroma_cens                                     ... tonnetz      \
statistics    kurtosis                                     ...     std       
number              01  02  03  04  05  06  07  08  09  10 ...      04  05   
1                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   
2                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   
3                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   
4                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   
5                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   
6                  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...     NaN NaN   

feature             zcr                               
statistics     kurtosis max mean median min skew std  
number      06       01  01   01     01  01   01  01  
1          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  
2          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  
3          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  
4          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  
5          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  
6          NaN      NaN NaN  NaN    NaN NaN  NaN NaN  

[6 rows x 518 columns]

In [8]:
def compute_features1(base_path):
    
    for root, dirnames, filenames in os.walk(base_path):
            row = 1
            for filename in filenames: 
                print(filename)
                filepath=os.path.join(root, filename ).replace('\\', '/')
     
                features = pd.Series(index=columns(), dtype=np.float32)
                
                def sub_features(name, values):
#                     
                    features[name, 'mean'] = np.mean(values, axis=1)
                    features[name, 'std'] = np.std(values, axis=1)
                    features[name, 'skew'] = stats.skew(values, axis=1)
                    features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
                    features[name, 'median'] = np.median(values, axis=1)
                    features[name, 'min'] = np.min(values, axis=1)
                    features[name, 'max'] = np.max(values, axis=1)

                x, sr = librosa.load(filepath, sr=None, mono=True)

            # """ZERO CROSSING RATE 
            #     The rate at which the signal changes from positive to negative or back.
            #     Key feature to classify percussive sounds.
            #     Percussion sounds: resonant, hyper-resonant, stony dull or dull.
            # """
                zcr = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
                sub_features("zcr", zcr)

            # 
            #    Losely relates to the twelve different pitch classes.

            #    CHROMA CQT - Constant Q
            #    It converts data series to frequency domain

            #    CHROMA CENS




            # """
                cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12, n_bins=7*12, tuning=None))
                assert cqt.shape[0] == 7 * 12
                assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

                chroma_cqt = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
                sub_features('chroma_cqt', chroma_cqt)

                chroma_cens = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
                sub_features('chroma_cens', chroma_cens)

                del cqt

            # """TONNETZ

            # """
                tonnetz = librosa.feature.tonnetz(chroma=chroma_cens)
                sub_features('tonnetz', tonnetz)

            # """ stft
            # """
                stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
                assert stft.shape[0] == 1 + 2048 // 2
                assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
                del x


                chroma_stft = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
                sub_features('chroma_stft', chroma_stft)

                rmse = librosa.feature.rmse(S=stft)
                sub_features('rmse', rmse)

                f = librosa.feature.spectral_centroid(S=stft)
                sub_features('spectral_centroid', f)
                f = librosa.feature.spectral_bandwidth(S=stft)
                sub_features('spectral_bandwidth', f)
                f= librosa.feature.spectral_contrast(S=stft, n_bands=6)
                sub_features('spectral_contrast', f)
                f = librosa.feature.spectral_rolloff(S=stft)
                sub_features('spectral_rolloff', f)

                mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
                del stft
                f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
                sub_features('mfcc', f)
                
                final.loc[row] = features
                print(final)
                print("))))))))))))))))))))))))))))))",row)
                row+=1
    
    return features, final


In [ ]:
x, sr = librosa.load("C:/Users/spari/Projects/DM Pro/fma-master/well/hiphop.00000.au", sr=None, mono=True)

In [ ]:
x

In [9]:
ff, df = compute_features1("C:/Users/spari/Projects/DM Pro/fma-master/raw")
#    

1.mp3
feature    chroma_cens                                                \
statistics    kurtosis                                                 
number              01        02       03       04        05      06   
1            -0.430685 -0.385172 -0.66519 -0.68003 -1.104885 -0.8939   
2                  NaN       NaN      NaN      NaN       NaN     NaN   
3                  NaN       NaN      NaN      NaN       NaN     NaN   
4                  NaN       NaN      NaN      NaN       NaN     NaN   
5                  NaN       NaN      NaN      NaN       NaN     NaN   
6                  NaN       NaN      NaN      NaN       NaN     NaN   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
1          -0.987085  0.936357 -0.144906 -0.400392    ...     0.162603   
2                NaN       NaN       NaN       Na

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
1            -0.430685 -0.385172 -0.665190 -0.680030 -1.104885 -0.893900   
2            -0.876122  0.753612 -0.222502 -1.011618 -1.140561 -0.606429   
3            -0.044490 -0.165290  0.137298 -0.268882  0.448512  0.213933   
4             0.541193  0.388908  0.949446 -0.101305  0.351690  0.390352   
5             0.559668 -1.026595  0.457561 -0.561433 -1.276582 -1.067090   
6                  NaN       NaN       NaN       NaN       NaN       NaN   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
1          -0.987085  0.936357 -0.144906 -0.400392    ...     0.162603   
2          -1.10194

In [11]:
#1 11 2 22 3 33
df

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
1            -0.430685 -0.385172 -0.665190 -0.680030 -1.104885 -0.893900   
2            -0.876122  0.753612 -0.222502 -1.011618 -1.140561 -0.606429   
3            -0.044490 -0.165290  0.137298 -0.268882  0.448512  0.213933   
4             0.541193  0.388908  0.949446 -0.101305  0.351690  0.390352   
5             0.559668 -1.026595  0.457561 -0.561433 -1.276582 -1.067090   
6             0.521713  1.726118  1.075138  0.388404 -0.374901  0.396010   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
1          -0.987085  0.936357 -0.144906 -0.400392    ...     0.162603   
2          -1.101946 -1.138608  3.194284  2.986508    ...     0.163773   
3          -1.057411 -1.145544 -0.413427 -0.627509    ...     0.068522   
4          -0.466126 -0.927716 -0.470822 -0.420819    ...     0.122073   
5           0.125427  0.559649 -0.478850  1.827161    ...     0.165512   
6           0.279743  0.160077  0.512151  0.096030    ...     0.082672   

feature                               zcr                                     \
statistics                       kurtosis       max      mean    median  min   
number            05        06         01        01        01        01   01   
1           0.024227  0.024270  13.830725  0.295410  0.041230  0.033691  0.0   
2           0.020849  0.027692   8.795350  0.310547  0.055778  0.052246  0.0   
3           0.017172  0.016784   8.181209  0.429199  0.061733  0.048340  0.0   
4           0.016928  0.022285   3.788705  0.406250  0.083925  0.068848  0.0   
5           0.022622  0.027730   3.709884  0.386719  0.066262  0.055664  0.0   
6           0.016246  0.017244   2.176194  0.380371  0.079003  0.077637  0.0   

feature                         
statistics      skew       std  
number            01        01  
1           3.453445  0.031889  
2           2.106095  0.032967  
3           2.447968  0.050560  
4           1.815920  0.071145  
5           1.521573  0.044533  
6           0.773747  0.036928  

[6 rows x 518 columns]

CHECKING DISTANCE OF TRAINING FEATURES BETWEEN SONGS

In [28]:
s1 = df.iloc[0]
s2 = df.iloc[1]
s3 = df.iloc[2]
s4 = df.iloc[3]
s5 = df.iloc[4]
s6 = df.iloc[5]

In [20]:
np.linalg.norm(s1-s2)

1620.7955

In [23]:
np.linalg.norm(s1-s3)

1524.5634

In [25]:
np.linalg.norm(s1-s4)

2188.3079

In [26]:
np.linalg.norm(s1-s5)

2823.8669

In [27]:
np.linalg.norm(s1-s6)

1861.9526

In [ ]:
# s1= df[1]
# # s2 = df[1]
# # sd

In [ ]:
df.to_csv('raw_dataframe.csv')

In [ ]:
np.linalg.norm(df.iloc[6]- df.iloc[1])

In [ ]:
# ff.sort_index(axis=0, inplace=True)
# # ff.sort_index(/axis=1, inplace=True)
# ff.to_csv('sampleee_features.csv', float_format='%.{}e'.format(10))